In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace
from network.lstm_deepfake import LSTMDF

from efficientnet_pytorch import EfficientNet

from pytorchcv.model_provider import get_model
import timm
from catalyst.dl import SupervisedRunner
from catalyst.contrib.nn import OneCycleLRWithWarmup


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


In [2]:
y_train = pd.read_csv(r'data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [3]:
#%%script false

batch_size = 32

x_train = load_img_dataset(r'data\least', batch_size, resize=None, crop=224, num_samples=100000)
x_val = load_img_val_dataset(r'data\img_val', batch_size)

In [4]:
#model = EfficientNet.from_name('efficientnet-b0')
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)

#model = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
#model.classifier = nn.Linear(model.classifier.in_features, 1)

#model = get_model("resnet18", pretrained=True)
#model = models.resnext50_32x4d(pretrained=True) 
#model =  models.resnext101_32x8d(pretrained=True) 
#model =  MyResNeXt()
#model = models.resnet18(pretrained=True)
#model._fc = nn.Linear(model._fc.in_features, 1)
#model.fc = nn.Linear(model.fc.in_features, 1)

Loaded pretrained weights for efficientnet-b0


In [5]:
%%script false
found = False
for child in model.named_parameters():  
    #'''
    if child[0] == 'layer4.0.conv2.weight':
        found = True
    #'''
    child[1].requires_grad = found

Couldn't find program: 'false'


In [6]:
model = model.to(device)
epochs = 10
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.) 
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 0.001, 0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(x_train), epochs=epochs)

#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: epoch // 30, lambda epoch: 0.95 ** epoch], last_epoch=-1)

In [7]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['_conv_stem.weight',
 '_bn0.weight',
 '_bn0.bias',
 '_blocks.0._depthwise_conv.weight',
 '_blocks.0._bn1.weight',
 '_blocks.0._bn1.bias',
 '_blocks.0._se_reduce.weight',
 '_blocks.0._se_reduce.bias',
 '_blocks.0._se_expand.weight',
 '_blocks.0._se_expand.bias',
 '_blocks.0._project_conv.weight',
 '_blocks.0._bn2.weight',
 '_blocks.0._bn2.bias',
 '_blocks.1._expand_conv.weight',
 '_blocks.1._bn0.weight',
 '_blocks.1._bn0.bias',
 '_blocks.1._depthwise_conv.weight',
 '_blocks.1._bn1.weight',
 '_blocks.1._bn1.bias',
 '_blocks.1._se_reduce.weight',
 '_blocks.1._se_reduce.bias',
 '_blocks.1._se_expand.weight',
 '_blocks.1._se_expand.bias',
 '_blocks.1._project_conv.weight',
 '_blocks.1._bn2.weight',
 '_blocks.1._bn2.bias',
 '_blocks.2._expand_conv.weight',
 '_blocks.2._bn0.weight',
 '_blocks.2._bn0.bias',
 '_blocks.2._depthwise_conv.weight',
 '_blocks.2._bn1.weight',
 '_blocks.2._bn1.bias',
 '_blocks.2._se_reduce.weight',
 '_blocks.2._se_reduce.bias',
 '_blocks.2._se_expand.weight',
 '_bloc

In [8]:
# %%script false
# 38% / 47% / 57% /58.5% / 61% - 0.656% (0.99344%) - video
# 83% undersampling / 50% oversampling / 89.8% original dataset / 0.08228

metric, lost = train_img(model, optimizer, scheduler, x_train, x_val, y_train, accuracy_b, device, 
                         [log_loss_sigmoid, accuracy_sigmoid_mean], 
                         epochs=epochs,
                         batch_size=batch_size, 
                         useScheduler=True,
                         checkpoint=0.35
                        )


Validation: metrics  0.7974275144110815 loss  0.4415668336651512
Train: metrics  0.76877 loss  0.4601346098613739
Expected LB value: 0.5238468336651512 0.5320082333315075
Epoch: 1



Validation: metrics  0.8179901918609653 loss  0.40540672625521357
Train: metrics  0.86093 loss  0.3081786439466476
Expected LB value: 0.4876867262552136 0.4884418388617031
Epoch: 2



Validation: metrics  0.828615675815194 loss  0.40977029181617164
Train: metrics  0.88841 loss  0.25674152747750284
Expected LB value: 0.49205029181617166 0.4936991467664718
Epoch: 3



Validation: metrics  0.8063322722188764 loss  0.5165050580195745
Train: metrics  0.90227 loss  0.22803649896621703
Expected LB value: 0.5987850580195745 0.6222952506259933
Epoch: 4


KeyboardInterrupt: 